In [8]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib.animation as animation
np.random.seed(0)

In [9]:
class GenDataLoader():
    def __init__(self, batch_size = 64, size = 200, low = -8, high = 8):
        self.batch_size = batch_size
        self.size = size
        self.low = low
        self.high = high
        self.range = 8
    def next_batch(self):
        # return np.reshape(np.linspace(-self.range, self.range, self.size) + \
        #        np.random.random(self.size) * 0.01, [1, self.size])
        return np.random.uniform(self.low, self.high, [self.batch_size, self.size]) #(batch_size, size)

In [10]:
class RealDataLoader():
    def __init__(self, batch_size = 64, size = 200, mu = 4, sigma = 0.5):
        self.batch_size = batch_size
        self.size = size
        self.mu = mu
        self.sigma = sigma

    def next_batch(self):

        data = np.random.normal(self.mu, self.sigma, [self.batch_size ,self.size])  #(batch_size, size)
        # data = np.random.normal(self.mu, self.sigma, [1, self.size])  #(batch_size, size)
        # data.sort()
        # for index, element in enumerate(data):
        #      data[index] = sorted(element)
        return data

In [11]:
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)

In [12]:
class Generator():
    def __init__(self, input_size = 200, hidden_size = 100, output_size = 200):
        with tf.variable_scope("generator"):
            self.inputs = tf.placeholder(tf.float32, shape=[None, input_size])
            weight1 = weight_variable(shape=[input_size, hidden_size], name="weight1") #(size, 100)
            bias1 = bias_variable(shape=[1, hidden_size], name="bias1") #(1, 100)
            output1 = tf.nn.relu(tf.matmul(self.inputs, weight1) + bias1, name="output1")   #(batch_size, 100)
            weight2 = weight_variable(shape=[hidden_size, output_size], name="weight2")
            bias2 = bias_variable(shape=[1, output_size], name="bias2")
            self.final_ouput = tf.nn.relu(tf.matmul(output1, weight2) + bias2, name="final_ouput")
            # self.final_ouput = tf.matmul(output1, weight2) + bias2
            #self.discriminator_out = tf.placeholder(tf.float32, shape=[None, 1])
            # self.loss = tf.reduce_mean(tf.log(1 - self.discriminator_out))
            # self.train_op = tf.train.AdamOptimizer(0.2).minimize(self.loss)

    def generate(self, sess, x):
        return sess.run(self.final_ouput, feed_dict={self.inputs:x})

    # def update(self, sess, discriminator, gen_samples):
    #     discriminator.discriminate(gen_samples)
    #     loss, _ = sess.run([self.loss, self.train_op], feed_dict={self.discriminator_out:discriminator_out})
    #     return loss

In [13]:
class Discriminator():
    def __init__(self, inputs, input_size = 200, hidden_size = 100):
        self.inputs = inputs
        with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
            # self.inputs = tf.placeholder(tf.float32, shape=[None, input_size])
            weight1 = weight_variable(shape=[input_size, hidden_size], name="weight1") #(size, 100)
            bias1 = bias_variable(shape=[1, hidden_size], name="bias1") #(1, 100)
            output1 = tf.nn.relu(tf.matmul(self.inputs, weight1) + bias1, name="output1")   #(batch_size, 100)
            weight2 = weight_variable(shape=[hidden_size, 1], name="weight2")
            bias2 = bias_variable(shape=[1, 1], name="bias2")
            self.final_output = tf.nn.sigmoid(tf.matmul(output1, weight2) + bias2, name="final_ouput") #range()
            # self.loss_g = -tf.reduce_mean(tf.log(self.real_discriminator_out)) - tf.reduce_mean(tf.log(1 - self.gen_discriminator_out))
            # self.train_op = tf.train.AdamOptimizer(0.2).minimize(self.loss)

    def discriminate(self, sess, x):
        return sess.run(self.final_output, feed_dict={self.inputs:x})

    # def update(self, sess, gen_discriminator_out, real_discriminator_out):
    #     loss, _ = sess.run([self.loss, self.train_op], feed_dict={self.gen_discriminator_out:gen_discriminator_out, self.real_discriminator_out:self.real_discriminator_out})
    #     return loss

In [16]:
if __name__ == '__main__':
    epoch = 1000
    batch_size = 1
    input_size = 200
    gen_data_loader = GenDataLoader(batch_size=1)
    # gen_data = gen_data_loader.next_batch()
    real_data_loader = RealDataLoader(batch_size=1)
    # real_data = gen_data_loader.next_batch()
    generator = Generator()
    real_inputs = tf.placeholder(tf.float32, shape=[None, input_size])
    gen_discriminator = Discriminator(generator.final_ouput)
    real_discriminator = Discriminator(real_inputs)
    loss_g = tf.reduce_mean(tf.log(1-gen_discriminator.final_output))
    loss_d = -tf.reduce_mean(tf.log(real_discriminator.final_output) + tf.log(1 - gen_discriminator.final_output))
    batch = tf.Variable(0)
    fig = plt.figure(1)
    learning_rate = tf.train.exponential_decay(
        0.001,  # Base learning rate.
        batch,  # Current index into the dataset.
        epoch // 4,  # 指数衰减（解决学习率）# Decay step - this decays 4 times throughout training process.
        0.95,  # Decay rate.
        staircase=True)
    train_op_g = tf.train.MomentumOptimizer(learning_rate, 0.6).minimize(loss_g)
    train_op_d = tf.train.MomentumOptimizer(learning_rate, 0.6).minimize(loss_d)
    real_data = real_data_loader.next_batch()
    real_data_plot_list = list()
    gen_samples_plot_list = list()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(epoch):
            for k in range(1):
                gen_data = gen_data_loader.next_batch()
                gen_samples = generator.generate(sess, gen_data)

                # gen_discrimator_out = gen_discriminator.discriminate(sess, gen_samples)
                # generator_loss = generator.update(sess, gen_discrimator_out)
                generator_loss, _ = sess.run([loss_g, train_op_g], feed_dict={generator.inputs:gen_data})
                # print("generator_loss:", generator_loss)
            for k in range(3):
                gen_data = gen_data_loader.next_batch()
                # real_data = real_data_loader.next_batch()
                gen_samples = generator.generate(sess, gen_data)
                discriminator_loss, _, real_dis_out, gen_dis_out = sess.run([loss_d, train_op_d, real_discriminator.final_output, gen_discriminator.final_output], feed_dict={generator.inputs:gen_data, real_inputs:real_data})
                print("real:", real_dis_out)
                print("gen:", gen_dis_out)
                # gen_discrimator_out = discriminator.discriminate(sess, gen_samples)
                # real_discrimator_out = discriminator.discriminate(sess, real_data)
                # discriminator_loss = discriminator.update(sess, gen_discrimator_out, real_discrimator_out)
                # print("discriminator_loss:", discriminator_loss)
            if (i % 50):
                gen_data = gen_data_loader.next_batch()
                # real_data = real_data_loader.next_batch()
                gen_samples = generator.generate(sess, gen_data)
                gen_samples_plot_list.append(gen_samples[0])
                real_data_plot_list.append(real_data[0])
                # plt.draw()  # 注意此函数需要调用



        x = np.linspace(1, 200, 200)
        def animate(i):
            plt.clf()
            plt.plot(x, real_data[0], color="red", label="real")
            plt.plot(x, gen_samples_plot_list[i], color="green", label="gen")

        ani = animation.FuncAnimation(fig, animate, interval=1000)
        plt.show()

real: [[0.0054143]]
gen: [[0.27587086]]
real: [[0.06333819]]
gen: [[0.2708394]]
real: [[0.7311959]]
gen: [[0.2133583]]
real: [[0.9763184]]
gen: [[0.0543206]]
real: [[0.9953513]]
gen: [[0.00517302]]
real: [[0.9982759]]
gen: [[0.13605279]]
real: [[0.99905246]]
gen: [[0.06042592]]
real: [[0.99933964]]
gen: [[0.05170571]]
real: [[0.999469]]
gen: [[0.01080175]]
real: [[0.9995347]]
gen: [[0.03210397]]
real: [[0.99957055]]
gen: [[0.28454676]]
real: [[0.9995912]]
gen: [[0.02279536]]
real: [[0.99960333]]
gen: [[0.27027193]]
real: [[0.99961096]]
gen: [[0.00316446]]
real: [[0.99961567]]
gen: [[0.02652642]]
real: [[0.99961877]]
gen: [[0.05944005]]
real: [[0.99962103]]
gen: [[0.01691937]]
real: [[0.9996227]]
gen: [[0.13766095]]
real: [[0.999624]]
gen: [[0.07951456]]
real: [[0.9996251]]
gen: [[0.00628651]]
real: [[0.99962616]]
gen: [[0.07438283]]
real: [[0.999627]]
gen: [[0.04325033]]
real: [[0.9996278]]
gen: [[0.08180363]]
real: [[0.99962866]]
gen: [[0.01973901]]
real: [[0.9996295]]
gen: [[0.077407

real: [[0.9997427]]
gen: [[0.00013442]]
real: [[0.99974316]]
gen: [[0.01555831]]
real: [[0.9997435]]
gen: [[0.00076597]]
real: [[0.9997439]]
gen: [[0.00040791]]
real: [[0.99974424]]
gen: [[0.00783955]]
real: [[0.9997446]]
gen: [[0.00047599]]
real: [[0.9997451]]
gen: [[0.00118341]]
real: [[0.9997454]]
gen: [[0.00286655]]
real: [[0.9997458]]
gen: [[4.7912243e-05]]
real: [[0.99974614]]
gen: [[0.00088432]]
real: [[0.9997465]]
gen: [[0.00029427]]
real: [[0.99974686]]
gen: [[0.01204232]]
real: [[0.9997472]]
gen: [[0.00035707]]
real: [[0.9997476]]
gen: [[0.00165811]]
real: [[0.99974793]]
gen: [[0.00045434]]
real: [[0.9997484]]
gen: [[0.00111558]]
real: [[0.99974877]]
gen: [[0.00026374]]
real: [[0.9997491]]
gen: [[0.00297927]]
real: [[0.9997495]]
gen: [[0.00068608]]
real: [[0.99974984]]
gen: [[0.00480113]]
real: [[0.9997502]]
gen: [[0.00183014]]
real: [[0.99975055]]
gen: [[0.00026302]]
real: [[0.9997509]]
gen: [[0.00129396]]
real: [[0.99975127]]
gen: [[0.00167224]]
real: [[0.9997516]]
gen: [[0

real: [[0.9998036]]
gen: [[0.00025787]]
real: [[0.99980384]]
gen: [[0.00139178]]
real: [[0.9998041]]
gen: [[0.02725015]]
real: [[0.9998043]]
gen: [[2.5467156e-05]]
real: [[0.99980456]]
gen: [[0.00047638]]
real: [[0.9998048]]
gen: [[0.00031625]]
real: [[0.99980503]]
gen: [[0.00733083]]
real: [[0.99980515]]
gen: [[0.00171545]]
real: [[0.9998054]]
gen: [[0.00379557]]
real: [[0.9998056]]
gen: [[0.00200716]]
real: [[0.99980587]]
gen: [[0.00121464]]
real: [[0.9998061]]
gen: [[1.8860503e-05]]
real: [[0.99980634]]
gen: [[0.00037555]]
real: [[0.99980646]]
gen: [[0.02736018]]
real: [[0.9998067]]
gen: [[0.00174715]]
real: [[0.99980694]]
gen: [[0.00062313]]
real: [[0.9998072]]
gen: [[0.00043744]]
real: [[0.9998074]]
gen: [[0.00029896]]
real: [[0.99980766]]
gen: [[0.00043903]]
real: [[0.9998078]]
gen: [[0.0012519]]
real: [[0.999808]]
gen: [[0.00021846]]
real: [[0.99980825]]
gen: [[0.00285484]]
real: [[0.9998085]]
gen: [[0.00246682]]
real: [[0.9998087]]
gen: [[0.00621448]]
real: [[0.99980885]]
gen: 

real: [[0.99984086]]
gen: [[0.00511391]]
real: [[0.999841]]
gen: [[0.00076953]]
real: [[0.9998412]]
gen: [[8.603944e-05]]
real: [[0.99984133]]
gen: [[0.00054229]]
real: [[0.99984145]]
gen: [[0.00041858]]
real: [[0.9998416]]
gen: [[0.00061336]]
real: [[0.9998418]]
gen: [[0.00393065]]
real: [[0.9998419]]
gen: [[0.00030313]]
real: [[0.99984205]]
gen: [[0.00011011]]
real: [[0.99984217]]
gen: [[0.00053676]]
real: [[0.9998423]]
gen: [[0.00583526]]
real: [[0.9998425]]
gen: [[0.0064128]]
real: [[0.99984264]]
gen: [[0.00012413]]
real: [[0.99984276]]
gen: [[5.993735e-05]]
real: [[0.9998429]]
gen: [[0.0029091]]
real: [[0.9998431]]
gen: [[0.0002031]]
real: [[0.99984324]]
gen: [[0.00047687]]
real: [[0.99984336]]
gen: [[3.703043e-05]]
real: [[0.9998435]]
gen: [[3.3840304e-05]]
real: [[0.9998436]]
gen: [[0.00140764]]
real: [[0.99984384]]
gen: [[0.00023364]]
real: [[0.99984396]]
gen: [[0.00021118]]
real: [[0.9998441]]
gen: [[0.00010276]]
real: [[0.9998442]]
gen: [[0.00021747]]
real: [[0.9998443]]
gen:

real: [[0.99986637]]
gen: [[0.00018059]]
real: [[0.9998665]]
gen: [[4.3866614e-05]]
real: [[0.9998666]]
gen: [[0.00133295]]
real: [[0.9998666]]
gen: [[0.0007879]]
real: [[0.9998667]]
gen: [[0.00083402]]
real: [[0.99986684]]
gen: [[0.00074412]]
real: [[0.99986696]]
gen: [[0.00040322]]
real: [[0.9998671]]
gen: [[0.0002378]]
real: [[0.9998672]]
gen: [[0.00206488]]
real: [[0.9998673]]
gen: [[0.00091497]]
real: [[0.99986744]]
gen: [[0.00181044]]
real: [[0.99986744]]
gen: [[0.00016128]]
real: [[0.99986756]]
gen: [[0.004345]]
real: [[0.9998677]]
gen: [[0.0001548]]
real: [[0.9998678]]
gen: [[4.6663004e-05]]
real: [[0.9998679]]
gen: [[0.00105156]]
real: [[0.99986804]]
gen: [[0.00137179]]
real: [[0.99986815]]
gen: [[0.00127045]]
real: [[0.99986815]]
gen: [[0.00012047]]
real: [[0.9998683]]
gen: [[0.00025178]]
real: [[0.9998684]]
gen: [[0.00047507]]
real: [[0.9998685]]
gen: [[4.3486714e-05]]
real: [[0.99986863]]
gen: [[0.00022971]]
real: [[0.99986875]]
gen: [[0.00014683]]
real: [[0.99986887]]
gen:

...

real: [[0.9999374]]
gen: [[3.3122877e-05]]
real: [[0.9999374]]
gen: [[0.00046497]]
real: [[0.9999374]]
gen: [[0.00120907]]
real: [[0.9999374]]
gen: [[1.5327627e-05]]
real: [[0.99993753]]
gen: [[0.00212787]]
real: [[0.99993753]]
gen: [[0.00010877]]
real: [[0.99993753]]
gen: [[0.00060523]]
real: [[0.99993753]]
gen: [[0.00083464]]
real: [[0.99993753]]
gen: [[2.166188e-05]]
real: [[0.99993753]]
gen: [[5.1492916e-05]]
real: [[0.99993765]]
gen: [[9.250357e-05]]
real: [[0.99993765]]
gen: [[5.1740506e-05]]
real: [[0.99993765]]
gen: [[0.00011728]]
real: [[0.99993765]]
gen: [[0.00010083]]
real: [[0.99993765]]
gen: [[0.00010111]]
real: [[0.9999378]]
gen: [[5.0758695e-06]]
real: [[0.9999378]]
gen: [[2.7653374e-05]]
real: [[0.9999378]]
gen: [[5.1153154e-05]]
real: [[0.9999378]]
gen: [[4.145928e-05]]
real: [[0.9999378]]
gen: [[2.5371499e-05]]
real: [[0.9999379]]
gen: [[0.00017535]]
real: [[0.9999379]]
gen: [[0.00079714]]
real: [[0.9999379]]
gen: [[0.00068533]]
real: [[0.9999379]]
gen: [[0.00010789]]

real: [[0.9999418]]
gen: [[0.00020073]]
real: [[0.99994195]]
gen: [[4.4768425e-05]]
real: [[0.99994195]]
gen: [[0.00193339]]
real: [[0.99994195]]
gen: [[4.6081448e-05]]
real: [[0.99994195]]
gen: [[0.00041867]]
real: [[0.99994195]]
gen: [[0.00011357]]
real: [[0.99994195]]
gen: [[1.6285783e-05]]
real: [[0.99994206]]
gen: [[0.00027751]]
real: [[0.99994206]]
gen: [[1.21867815e-05]]
real: [[0.99994206]]
gen: [[1.6600226e-05]]
real: [[0.99994206]]
gen: [[3.6559475e-05]]
real: [[0.99994206]]
gen: [[1.0808248e-05]]
real: [[0.99994206]]
gen: [[0.00051976]]
real: [[0.9999422]]
gen: [[1.5205143e-05]]
real: [[0.9999422]]
gen: [[0.00062998]]
real: [[0.9999422]]
gen: [[0.00181346]]
real: [[0.9999422]]
gen: [[0.00077492]]
real: [[0.9999422]]
gen: [[5.583469e-05]]
real: [[0.9999422]]
gen: [[0.00034871]]
real: [[0.9999423]]
gen: [[0.00602732]]
real: [[0.9999423]]
gen: [[8.375864e-05]]
real: [[0.9999423]]
gen: [[0.00053485]]
real: [[0.9999423]]
gen: [[3.900086e-05]]
real: [[0.9999423]]
gen: [[8.316761e-

real: [[0.99994564]]
gen: [[1.907715e-06]]
real: [[0.99994564]]
gen: [[0.00035498]]
real: [[0.99994564]]
gen: [[2.611221e-05]]
real: [[0.99994564]]
gen: [[0.00010898]]
real: [[0.99994576]]
gen: [[1.4270186e-05]]
real: [[0.99994576]]
gen: [[3.463624e-05]]
real: [[0.99994576]]
gen: [[0.00037505]]
real: [[0.99994576]]
gen: [[2.1568212e-05]]
real: [[0.99994576]]
gen: [[6.428165e-05]]
real: [[0.99994576]]
gen: [[9.7000186e-05]]
real: [[0.99994576]]
gen: [[0.00041534]]
real: [[0.9999459]]
gen: [[9.004936e-05]]
real: [[0.9999459]]
gen: [[1.1721104e-05]]
real: [[0.9999459]]
gen: [[3.0555653e-05]]
real: [[0.9999459]]
gen: [[2.3657587e-05]]
real: [[0.9999459]]
gen: [[0.00064506]]
real: [[0.9999459]]
gen: [[7.984779e-05]]
real: [[0.9999459]]
gen: [[0.0001275]]
real: [[0.999946]]
gen: [[8.941436e-06]]
real: [[0.999946]]
gen: [[8.530492e-06]]
real: [[0.999946]]
gen: [[2.7457314e-05]]
real: [[0.999946]]
gen: [[5.331823e-05]]
real: [[0.999946]]
gen: [[0.00027088]]
real: [[0.999946]]
gen: [[0.00018504

real: [[0.999949]]
gen: [[0.00140299]]
real: [[0.999949]]
gen: [[0.00044931]]
real: [[0.999949]]
gen: [[3.895552e-06]]
real: [[0.999949]]
gen: [[5.410269e-05]]
real: [[0.9999491]]
gen: [[1.61786e-05]]
real: [[0.9999491]]
gen: [[2.6459815e-05]]
real: [[0.9999491]]
gen: [[3.6906033e-06]]
real: [[0.9999491]]
gen: [[1.4675586e-05]]
real: [[0.9999491]]
gen: [[2.3226652e-05]]
real: [[0.9999491]]
gen: [[2.337912e-05]]
real: [[0.9999491]]
gen: [[0.00166069]]
real: [[0.9999491]]
gen: [[1.6472959e-05]]
real: [[0.9999492]]
gen: [[1.5805486e-05]]
real: [[0.9999492]]
gen: [[0.00124151]]
real: [[0.9999492]]
gen: [[3.520102e-05]]
real: [[0.9999492]]
gen: [[0.00209352]]
real: [[0.9999492]]
gen: [[0.00050873]]
real: [[0.9999492]]
gen: [[4.525053e-06]]
real: [[0.9999492]]
gen: [[4.5410047e-06]]
real: [[0.9999492]]
gen: [[2.6979233e-05]]
real: [[0.99994934]]
gen: [[0.00023597]]
real: [[0.99994934]]
gen: [[1.86621e-05]]
real: [[0.99994934]]
gen: [[0.01050296]]
real: [[0.99994934]]
gen: [[0.00029573]]
real

<Figure size 432x288 with 0 Axes>